<a href="https://colab.research.google.com/github/chadbr/ecimtest/blob/main/ecim_both.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Init

In [1]:
# google-api-core==2.19.1
# google-auth==2.33.0
# google-cloud-core==2.4.1
# google-cloud-storage==2.18.2
# google-crc32c==1.5.0
# google-resumable-media==2.7.2
# googleapis-common-protos==1.63.2

!pip install osdu-api==0.26.0 \
  --extra-index-url https://community.opengroup.org/api/v4/projects/148/packages/pypi/simple
!pip install requests==2.32.3
!pip install tenacity==9.0.0
!pip install urllib3==2.2.2



Looking in indexes: https://pypi.org/simple, https://community.opengroup.org/api/v4/projects/148/packages/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.1/155.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


### Required

In [2]:
import pprint
import requests

import io
import os
import pprint
import urllib3
import json

from pprint import pprint

from IPython.display import display
from PIL import Image

from osdu_api.auth.refresh_token import BaseTokenRefresher

from osdu_api.clients.storage.record_client import RecordClient
from osdu_api.clients.search.search_client import SearchClient
from osdu_api.clients.schema.schema_client import SchemaClient
from osdu_api.clients.dataset.dataset_dms_client import DatasetDmsClient
from osdu_api.clients.dataset.dataset_registry_client import DatasetRegistryClient

from osdu_api.model.storage.legal import Legal
from osdu_api.model.storage.record import Record
from osdu_api.model.storage.record_ancestry import RecordAncestry
from osdu_api.model.storage.acl import Acl
from osdu_api.model.search.query_request import QueryRequest
from osdu_api.model.storage.query_records_request import QueryRecordsRequest
from osdu_api.model.dataset.create_dataset_registries_request import CreateDatasetRegistriesRequest
from google.colab import userdata


urllib3.disable_warnings()
print("\ndone")


done


#### Option 1: Credentials for Cimpl


Credentials for remote Cimpl

In [ ]:
%%script echo skip the optional part

CLOUD_PROVIDER = "baremetal"
os.environ["KEYCLOAK_AUTH_URL"] = "https://keycloak.ecim24.com/realms/osdu/protocol/openid-connect/token"
os.environ["KEYCLOAK_CLIENT_ID"] = "datafier"
os.environ["KEYCLOAK_CLIENT_SECRET"] = userdata.get("datafierpassword")
os.environ["CLOUD_PROVIDER"] = CLOUD_PROVIDER

schema = 'https'
BASE_URL = f"{schema}://osdu.ecim24.com"

print("\ndone")

Credentials for local Cimpl

In [ ]:
# %%script echo skip the optional part

# CLOUD_PROVIDER = "baremetal"
# os.environ["KEYCLOAK_AUTH_URL"] = "http://keycloak.localhost/realms/osdu/protocol/openid-connect/token"
# os.environ["KEYCLOAK_CLIENT_ID"] = "osdu-admin"
# os.environ["KEYCLOAK_CLIENT_SECRET"] = ""
# os.environ["CLOUD_PROVIDER"] = CLOUD_PROVIDER

# schema = 'http'
# BASE_URL = f"{schema}://osdu.localhost"

# print("\ndone")

#### Option 2: Credentials for GC

In [3]:
# %%script echo skip the optional part

!gcloud auth application-default login --no-launch-browser


CLOUD_PROVIDER = "gc"

os.environ["CLOUD_PROVIDER"] = CLOUD_PROVIDER

schema = 'https'
BASE_URL = f"{schema}://gcp.ecim24.com"


Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=La36zhXpSNMLZ7onQq5iFep6kF8zGo&prompt=consent&token_usage=remote&access_type=offline&code_challenge=RX_4HcD8jzkYG1NUlyeUQ7Y6gVQa206yIJql50jTPak&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AQlEd8xbkuWNSwdig49zb8HtC-ZMXbyu0m3arcR9dAG4y5HvlTJUC14-TcE3UFslQ-tV5g

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

#### Loading the config

In [4]:
config_url = BASE_URL + "/api/config/v1/postman-environment"
config_response = requests.get(config_url).json()
config = {}
for entity in config_response['values']:
    config[entity['key']] = entity['value']
config['data-partition-id'] = "osdu"

# pprint(config_response)
print(config['SEISMICDMS_HOST'])
print(f"data partition is {config['data-partition-id']}")
print("\ndone")

gcp.ecim24.com/api/seismic-store/v3
data partition is osdu

done


In [5]:
token_refresher = BaseTokenRefresher()
token_refresher.refresh_token()
#print("\ndone")

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


'ya29.a0AcM612xcaG6f47VceHFWtvMbU_QUNcjGlrq4tMUIqhpcmWA-pU_2IcjUPQy7A1aw58m8YQpmhzgrdQTa8f455WQqCh1VQq_wBTiq6Ix94U1VxRTnCoqP-G2uNLiYqRHYELIf99L5Sx2r3HU8lfQ7iisqaFfJuTtAaZc6BvOWaCgYKAZoSARASFQHGX2MisvBj9b7pajsVr8UeudkLXg0175'

#### OSDU clients and default values

In [6]:
search_client = SearchClient(
    search_url=f"{schema}://{config['SEARCH_HOST']}",
    provider=CLOUD_PROVIDER,
    token_refresher=token_refresher,
    data_partition_id=config['data-partition-id']
)

schema_client = SchemaClient(
    schema_url=f"{schema}://{config['SCHEMA_HOST']}/api/schema-service/v1",
    provider=CLOUD_PROVIDER,
    token_refresher=token_refresher,
    data_partition_id=config['data-partition-id']
)

dataset_client = DatasetDmsClient(
    dataset_url=f"{schema}://{config['DATASET_HOST']}",
    provider=CLOUD_PROVIDER,
    token_refresher=token_refresher,
    data_partition_id=config['data-partition-id']
)
dataset_registry_client = DatasetRegistryClient(
    dataset_url=f"{schema}://{config['DATASET_HOST']}",
    provider=CLOUD_PROVIDER,
    token_refresher=token_refresher,
    data_partition_id=config['data-partition-id']
)

storage_client = record_client = RecordClient(
    storage_url=f"{schema}://{config['STORAGE_HOST']}",
    provider=CLOUD_PROVIDER,
    token_refresher=token_refresher,
    data_partition_id=config['data-partition-id']
)

query_request = QueryRequest(
    kind="*:*:*:*",
    query="",
    limit=1,
    returned_fields=["*"]
)
response = search_client.query_records(query_request).json()
pprint(response)

default_acl = response['results'][0]['acl']
default_legal_tags = ["osdu-default-data-tag"]

pprint(response)
pprint(default_acl)
pprint(default_legal_tags)

urllib3.disable_warnings()

print("\ndone")

{'aggregations': None,
 'phraseSuggestions': [],
 'results': [{'acl': {'owners': ['data.default.owners@osdu.group'],
                      'viewers': ['data.default.viewers@osdu.group']},
              'authority': 'osdu',
              'createTime': '2024-09-06T06:06:53.214Z',
              'createUser': 'datafier@ecim-2024.iam.gserviceaccount.com',
              'data': {'Checksum': None,
                       'DatasetProperties.FileSourceInfo.Checksum': None,
                       'DatasetProperties.FileSourceInfo.ChecksumAlgorithm': None,
                       'DatasetProperties.FileSourceInfo.EncodingFormatTypeID': 'osdu:reference-data--EncodingFormatType:ASCII:',
                       'DatasetProperties.FileSourceInfo.FileSize': 15728640,
                       'DatasetProperties.FileSourceInfo.FileSource': '/8d8b4311-8d14-4a6f-b890-83f4d3c418cc/671e68e9836244048f608a761eb5adb6',
                       'DatasetProperties.FileSourceInfo.Name': 'BCU+ST0202R08_PS_PSDM_FULL_PP+ST

# Slot 1 - Using Core APIs

#### All to register and make successful connection to workbook and both OSDU instances

In [7]:
search_response = requests.get(f"{schema}://{config['SEARCH_HOST']}/info")

pprint(search_response.json())

print("\ndone")

{'artifactId': 'search-gc',
 'branch': 'v0.26.0',
 'buildTime': '2024-04-23T16:17:45.440Z',
 'commitId': '1880bd9b03ced4a585794e88664580d2e4f5786f',
 'commitMessage': 'OSDU Data Platform v0.26.0',
 'connectedOuterServices': [{'name': 'Redis-RedisCacheBuilder$2',
                             'version': '7.2.5'},
                            {'name': 'ElasticSearch-osdu',
                             'version': '7.17.23'}],
 'groupId': 'org.opengroup.osdu',
 'version': '0.26.0'}

done


#### All able to execute and understand the result on some of the core APIs

In [8]:
# List all records
query_request = QueryRequest(
    kind= "*:*:*:*",
    query="",
    limit=2,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint(response.json())

print("\ndone")

{'aggregations': None,
 'phraseSuggestions': [],
 'results': [{'acl': {'owners': ['data.default.owners@osdu.group'],
                      'viewers': ['data.default.viewers@osdu.group']},
              'authority': 'osdu',
              'createTime': '2024-09-06T06:06:53.214Z',
              'createUser': 'datafier@ecim-2024.iam.gserviceaccount.com',
              'data': {'Checksum': None,
                       'DatasetProperties.FileSourceInfo.Checksum': None,
                       'DatasetProperties.FileSourceInfo.ChecksumAlgorithm': None,
                       'DatasetProperties.FileSourceInfo.EncodingFormatTypeID': 'osdu:reference-data--EncodingFormatType:ASCII:',
                       'DatasetProperties.FileSourceInfo.FileSize': 15728640,
                       'DatasetProperties.FileSourceInfo.FileSource': '/8d8b4311-8d14-4a6f-b890-83f4d3c418cc/671e68e9836244048f608a761eb5adb6',
                       'DatasetProperties.FileSourceInfo.Name': 'BCU+ST0202R08_PS_PSDM_FULL_PP+ST

In [9]:
# List a specific subset of records based on schema type/kind
query_request = QueryRequest(
    kind="osdu:wks:master-data--Agreement:1.0.0",
    query="",
    limit=2,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint(response.json())

print("\ndone")

{'aggregations': None,
 'phraseSuggestions': [],
 'results': [{'acl': {'owners': ['data.default.owners@osdu.group'],
                      'viewers': ['data.default.viewers@osdu.group']},
              'authority': 'osdu',
              'createTime': '2024-09-05T21:34:13.195Z',
              'createUser': 'datafier@ecim-2024.iam.gserviceaccount.com',
              'data': {'AgreementExternalID': None,
                       'AgreementExternalSystem': None,
                       'AgreementIdentifier': None,
                       'AgreementName': 'WesternGeco',
                       'AgreementParentID': None,
                       'AgreementTypeID': None,
                       'ExistenceKind': None,
                       'ResourceCurationStatus': None,
                       'ResourceHomeRegionID': None,
                       'ResourceLifecycleStatus': None,
                       'ResourceSecurityClassification': None,
                       'Source': None,
                      

In [10]:
# Find the description of specific schema/kind to show that schemas are a part of the platform
response = schema_client.get_schema_by_id('osdu:wks:master-data--Agreement:1.0.0').json()
# String representation of the response for pretty printing
schema_json = json.dumps(response, indent=4)
print(schema_json)
print("\ndone")

{
    "x-osdu-inheriting-from-kind": [],
    "x-osdu-license": "Copyright 2022, The Open Group \\nLicensed under the Apache License, Version 2.0 (the \"License\"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 . Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.",
    "$schema": "http://json-schema.org/draft-07/schema#",
    "x-osdu-schema-source": "osdu:wks:master-data--Agreement:1.0.0",
    "description": "A contract or other covenant between Company and counterparties which is relevant to the data universe because it includes terms governing use of data.",
    "title": "Agreement",
    "type": "object",
    "required": [
        "

#### Show that the platform is generic and/extendable and can store any data type in addition to defined "well known" schemas






In [ ]:
response = dataset_client.storage_instructions(kind_sub_type='dataset--File.Generic').json()

pprint(response)

In [ ]:
# # get the file from Google Drive - OPTION #1

# from google.colab import drive
# drive.mount('/content/drive')

# with open('/content/drive/MyDrive/image.jpg', 'rb') as file_content:
#     file_content = file_content.read()


In [ ]:
# get a file from the bucket

from google.cloud import storage as google_storage

gs_client = google_storage.Client.create_anonymous_client()
bucket = gs_client.bucket("ecim-demo-2024")
blob = bucket.get_blob("image.jpg")
file_content = blob.download_as_bytes()

In [ ]:
# All to upload an image

def upload_file_and_metadata(file_content: bytes) -> str:
    storage_instruction = dataset_client.storage_instructions(kind_sub_type="dataset--File.Generic")
    storage_location = storage_instruction.json()['storageLocation']
    #### Uploading

    new_file = dataset_client.put_file(url=storage_location['signedUrl'], data=file_content, no_auth=True)

    record_list = [
                    Record( kind='osdu:wks:dataset--File.Generic:1.0.0',
                            acl=Acl(owners=default_acl['owners'], viewers=default_acl['viewers']),
                            legal=Legal(legaltags=default_legal_tags, other_relevant_data_countries=["US"], status="compliant"),
                            data =   {
                                        "DatasetProperties": {
                                            "FileSourceInfo": {
                                                "FileSource": storage_location['fileSource'],
                                                "Name": "image.jpg"
                                            }
                                        },
                                        "Name": "Dataset with images",
                                        "ResourceSecurityClassification": f"{config['data-partition-id']}:reference-data--ResourceSecurityClassification:RESTRICTED:",
                                        "SchemaFormatTypeID": f"{config['data-partition-id']}:reference-data--SchemaFormatType:TabSeparatedColumnarText:"
                                    },
                            ancestry=RecordAncestry(parents=[]))
                ]



    response = dataset_registry_client.register_dataset(CreateDatasetRegistriesRequest(dataset_registries=record_list))
    response.raise_for_status()
    registered_dataset = response.json()

    pprint(storage_location)
    pprint(registered_dataset)
    registered_dataset_id = registered_dataset["datasetRegistries"][0]["id"]
    return registered_dataset_id


dataset_id = upload_file_and_metadata(file_content)


In [ ]:
# Use dataset id to download the file
# Get retrieval instructions. Usually, we need a signed url from them

retrieval_instructions = dataset_client.retrieval_instructions(dataset_id)
retrieval_instructions.raise_for_status()
retrieval_instructions = retrieval_instructions.json()
signed_url = retrieval_instructions["datasets"][0]["retrievalProperties"]["signedUrl"]

# Get the file's content with the signed url and display it
response = requests.get(signed_url)
response.raise_for_status()
image_file = io.BytesIO(response.content)

display(Image.open(image_file))



In [ ]:
# All to find there image and probably the other attendances images
query_request = QueryRequest(
    kind="*:*:*:*",
    query="data.Name:\"Dataset with images\"",
    limit=10,
    returned_fields=["*"],
)
response = search_client.query_records(query_request)

pprint(response.json())

# Slot 2 - Investigating well data

#### All to search/list wells/wellbores/wellogs/trajectories/... that is uploaded in the **instances**

In [ ]:
# Wells
query_request = QueryRequest(
    kind="osdu:wks:master-data--Well:1.0.0", # "*:*:master-data--Well:*",
    query="",
    limit=5,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint(response.json())

In [ ]:
# Welllogs

query_request = QueryRequest(
    kind="osdu:wks:work-product-component--WellLog:*.*.*",
    query="",
    limit=5,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint(response.json())

In [ ]:
# Wellbores

query_request = QueryRequest(
    kind="osdu:wks:master-data--Wellbore:1.0.0",
    query="",
    limit=5,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint(response.json())

In [ ]:
# Trajectories

query_request = QueryRequest(
    kind="osdu:wks:work-product-component--WellboreTrajectory:*.*.*",
    query="",
    limit=5,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint(response.json())

#### For one of the few well/wellobres that we have prepped a bit

In [ ]:
# Use lineage to find welllogs/trajectories/... that belongs to a specific well/wellbore
# Welllogs

query_request = QueryRequest(
    kind="osdu:wks:work-product-component--WellLog:*.*.*",
    query="data.WellboreID:\"osdu:master-data--Wellbore:7233:\"",
    limit=5,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint(response.json())


In [ ]:
# By usage show that the platform can handle and store versions for some of these data
record = Record(
    kind='osdu:wks:dataset--File.Generic:1.0.0',
    acl=Acl(owners=default_acl['owners'], viewers=default_acl['viewers']),
    legal=Legal(legaltags=default_legal_tags, other_relevant_data_countries=["US"], status="compliant"),
    data= {
        "msg": "Initial Message from AutoTest while testing update",
        "weight": 777.0
    })
# create
response = storage_client.create_update_records(records=[record])
stored_record = response.json()
recordId = stored_record['recordIds'][0]
pprint("the new record")
pprint(stored_record)
print("\n")

# get versions
versions = storage_client.get_record_versions(recordId=recordId).json()
pprint("current versions")
pprint(versions)
print("\n")


# update
pprint("updating the record")
record.data["weight"] = 778
record.id = recordId
response = storage_client.create_update_records(records=[])

# get versions
versions = storage_client.get_record_versions(recordId=recordId).json()
pprint("current versions")
pprint(versions)
print("\n")

# update
pprint("updating the record")
record.data["weight"] = 779
record.id = recordId
response = storage_client.create_update_records(records=[record])

# get versions
versions = storage_client.get_record_versions(recordId=recordId).json()
pprint("current versions")
pprint(versions)
checking_version = versions["versions"][-1]
pprint("current version")
pprint(checking_version)
print("\n")

# get specific version
version = storage_client.get_specific_record(recordId=recordId, version=checking_version).json()
pprint("specific version")
pprint(version)

In [ ]:
# By usage show the difference between index/Storage/File APIs, for a data object
# In practice the user seeing the indexed data, the stored data and the payload data (CSV, LAS, DLIS, Parquet)

# upload file and metadata using dataset
upload_file_and_metadata(image_file)

# 10 sec for indexing
# search
query_request = QueryRequest(
    kind= "*:*:*:*",
    query="data.Name:\"Dataset with images\"",
    limit=2,
    returned_fields=["*"]
)
images_with_metadata = search_client.query_records(query_request).json()
pprint(images_with_metadata)

# get record from storage
record_id = images_with_metadata['results'][0]['id']
images_with_metadata = storage_client.query_record(recordId=record_id)
pprint(images_with_metadata.json())

# download using File Service
file_host = config["FILE_HOST"]
url = f'{file_host}/files/{record_id}/downloadURL'
downloadUrlObject = requests.get(url, headers={'Data-Partition-Id': config['data-partition-id'], 'Authorization': f'Bearer {token_refresher.access_token}'}).json()
pprint(downloadUrlObject)

file_response = requests.get(downloadUrlObject['SignedUrl'], headers={'Authorization': f'Bearer {token_refresher.access_token}'}, allow_redirects=True)
file_response.raise_for_status()
file_content = file_response.content
file_object = io.BytesIO(file_content)

print("\n", 24 * "*")
print("\nDisplay image:\n")
display(Image.open(file_object))



# Slot 3 - Wellbore DDMS

For one of the few well/wellbores that we have prepped a bit

In [ ]:
# By usage see that the DDMS is an added capability to Index/Storage/File
# By usage see that same data can be extracted through both core and DDMS
# Show examples on using some of the DDMS APIs. To be determined
# Get basic well info to show that it's the same info as you get through the core APIs like (well header, wellbore header, ...)
# Demonstrate use of some added functionality in the DDMS
# Most likely related to logs and extraction of 1 specific curve, a specific curve interval, etc.

# Create a new well

url = f"{schema}://{config['WELLBORE_DDMS_HOST']}/ddms/v3/wells"
headers = {'data-partition-id': config['data-partition-id'], 'Authorization': f'Bearer {token_refresher.access_token}'}
body = [{
        "acl": default_acl,
        "data": {
            "FacilityName": "Faciliity_AutoTest_999956686160",
            "ExtensionProperties": {
                "UWI": "20-000-00000-00"
            }
        },
        "id": f"{config['data-partition-id']}:master-data--Well:AutoTest_999956686160",
        "kind": "osdu:wks:master-data--Well:1.0.0",
        "legal": {
            "legaltags": default_legal_tags,
            "otherRelevantDataCountries": [
                "FR",
                "US"
            ],
            "status": "compliant"
        }
    }]


uploaded_well = requests.post(url, json=body, headers=headers).json()
pprint(uploaded_well)

# Get the well using Search
query_request = QueryRequest(
    kind="osdu:wks:master-data--Well:1.0.0",
    query="id:\"osdu:master-data--Well:AutoTest_999956686160\"",
    limit=5,
    returned_fields=["*"]
)
downloaded_well1 = search_client.query_records(query_request).json()
pprint(downloaded_well1)

# Get the well using Wellbore DDMS
url = f"{schema}://{config['WELLBORE_DDMS_HOST']}/ddms/v3/wells/{uploaded_well['recordIds'][0]}"
downloaded_well2 = requests.get(url, headers=headers).json()
pprint(downloaded_well2)


# Create a new wellbore

url = f"{schema}://{config['WELLBORE_DDMS_HOST']}/ddms/v3/wellbores"
headers = {'data-partition-id': config['data-partition-id'], 'Authorization': f'Bearer {token_refresher.access_token}'}
body = [{
        "acl": default_acl,
        "data": {
            "FacilityName": "Faciliity_AutoTest_999956686160",
            "ExtensionProperties": {
                "UWI": "20-000-00000-00"
            }
        },
        "id": f"{config['data-partition-id']}:master-data--Wellbore:AutoTest_999956686160",
        "kind": "osdu:wks:master-data--Wellbore:1.0.0",
        "legal": {
            "legaltags": default_legal_tags,
            "otherRelevantDataCountries": [
                "FR",
                "US"
            ]
        },
        "data": {
            "ExtensionProperties": {
                "airGap": {
                    "unitKey": "ft",
                    "value": 35.0
                },
                "country": "United States of America",
                "operator": "Francois Vinyes",
                "uwi": "33-089-00300-00-01",
                "wellboreType": "bypass"
            },
            "SpatialLocation": {
                "Wgs84Coordinates": {
                    "features": [{
                            "geometry": {
                                "coordinates": [
                                    [
                                        -103.2380248,
                                        46.8925081,
                                        5301.0
                                    ],
                                    [
                                        -103.2380248,
                                        46.8925081,
                                        2801.0
                                    ],
                                    [
                                        -103.2378748,
                                        46.892608100000004,
                                        301.0
                                    ],
                                    [
                                        -103.23742477750001,
                                        46.89270811,
                                        -2199.0
                                    ],
                                    [
                                        -103.23667470999663,
                                        46.892808120001,
                                        -4699.0
                                    ],
                                    [
                                        -103.2356245974865,
                                        46.892908130002,
                                        -7199.0
                                    ]
                                ],
                                "type": "LineString"
                            },
                            "properties": {
                                "name": "Newton 2-31-Lat-1"
                            },
                            "type": "Feature"
                        }
                    ],
                    "type": "FeatureCollection"
                }
            },
            "FacilityName": "Faciliity_AutoTest_99995668616",
            "WellID": "osdu:master-data--Well:AutoTest_999956686160:"
        },
        "meta": [{
                "kind": "Unit",
                "name": "Measure depth default unit",
                "persistableReference": "persistableReference",
                "propertyNames": ["symbol"]
            }
        ]
    }]

uploaded_wellbore = requests.post(url, json=body, headers=headers).json()
pprint(uploaded_wellbore)

# Get the wellbore using Search
query_request = QueryRequest(
    kind="osdu:wks:master-data--Wellbore:1.0.0",
    query="id:\"osdu:master-data--Wellbore:AutoTest_999956686160\"",
    limit=5,
    returned_fields=["*"]
)
downloaded_wellbore1 = search_client.query_records(query_request).json()
# pprint(downloaded_wellbore1)

# Get the wellbore using Wellbore DDMS
url = f"{schema}://{config['WELLBORE_DDMS_HOST']}/ddms/v3/wellbores/{uploaded_wellbore['recordIds'][0]}"
downloaded_wellbore2 = requests.get(url, headers=headers).json()
pprint(downloaded_wellbore2)

# Create a trajectory
url = f"{schema}://{config['WELLBORE_DDMS_HOST']}/ddms/v3/wellboretrajectories"
headers = {'data-partition-id': config['data-partition-id'], 'Authorization': f'Bearer {token_refresher.access_token}'}
body = [
    {
        "acl": default_acl,
        "legal": {
            "legaltags": default_legal_tags,
            "otherRelevantDataCountries": [
                "FR",
                "US"
            ]
        },
        "id": f"{config['data-partition-id']}:work-product-component--WellboreTrajectory:AutoTest_999956686160",
        "kind": f"{config['data-partition-id']}:wks:work-product-component--WellboreTrajectory:1.1.0",
        "data": {
            "Name": "Wellbore_Trajectory_AutoTest_999956686160",
            "WellboreID": f"{config['data-partition-id']}:master-data--Wellbore::AutoTest_999956686160:",
            "TopDepthMeasuredDepth": 0,
            "BaseDepthMeasuredDepth": 2.0,
            "VerticalMeasurement": {
                "EffectiveDateTime": "2021-08-17T14:13:08.174Z",
                "VerticalMeasurement": 0
            },
            "AvailableTrajectoryStationProperties": [
                {
                    "TrajectoryStationPropertyTypeID": f"{config['data-partition-id']}:reference-data--TrajectoryStationPropertyType:MD:",
                    "StationPropertyUnitID": f"{config['data-partition-id']}:reference-data--UnitOfMeasure:m:",
                    "Name": "Measured Depth"
                },
                {
                    "TrajectoryStationPropertyTypeID": f"{config['data-partition-id']}:reference-data--TrajectoryStationPropertyType:INCL:",
                    "StationPropertyUnitID": f"{config['data-partition-id']}:reference-data--UnitOfMeasure:dega:",
                    "Name": "Inclination"
                },
                {
                    "TrajectoryStationPropertyTypeID": f"{config['data-partition-id']}:reference-data--TrajectoryStationPropertyType:AZIM_TN:",
                    "StationPropertyUnitID": f"{config['data-partition-id']}:reference-data--UnitOfMeasure:dega:",
                    "Name": "Azimuth"
                },
                {
                    "TrajectoryStationPropertyTypeID": f"{config['data-partition-id']}:reference-data--TrajectoryStationPropertyType:TVD:",
                    "StationPropertyUnitID": f"{config['data-partition-id']}:reference-data--UnitOfMeasure:m:",
                    "Name": "True Vertical Depth"
                }
            ]
        }
    }
]
uploaded_trajectories = requests.post(url, json=body, headers=headers).json()
pprint(uploaded_trajectories)

# Get the trajectory using Search
query_request = QueryRequest(
    kind="osdu:wks:work-product-component--WellboreTrajectory:1.1.0",
    query="id:\"osdu:work-product-component--WellboreTrajectory:AutoTest_999956686160\"",
    limit=5,
    returned_fields=["*"]
)
downloaded_trajectories1 = search_client.query_records(query_request).json()
pprint(downloaded_trajectories1)

# Get the trajectory using Wellbore DDMS
url = f"{schema}://{config['WELLBORE_DDMS_HOST']}/ddms/v3/wellboretrajectories/{uploaded_trajectories['recordIds'][0]}"
downloaded_trajectories2 = requests.get(url, headers=headers).json()
pprint(downloaded_trajectories2)
